In [31]:
import gzip
import urllib.request as request
import pandas as pd
import io

In [32]:
def get_codes(dictionary):
      url_dict = "https://ec.europa.eu/eurostat/" + \
          "estat-navtree-portlet-prod/BulkDownloadListing" + \
          "?sort=1&downfile=dic%2Fen%2F" +\
          dictionary + ".dic"
      with request.urlopen(url_dict) as resp:
                file_content = resp.readlines()
      d={}
      for i in file_content:
            if len(i) > 1:
                  row=i.decode().split('\t')
                  d[row[0]] = row[1].strip()
      return d

In [33]:
def get_df(dataset):
    """ 
    dataset: choose the dataset code from:
https://ec.europa.eu/eurostat/databrowser/explore/all/all_themes?lang=en&display=list&sort=category
    something like EXT_LT_INTROEU27_2020
List of datasets for downloding: 
https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?dir=data&sort=1&sort=2&start=a
Returtn the cleaned dataset with decoded abbreviation

    """

    dataset = dataset.lower()
    url = "https://ec.europa.eu/eurostat/" + \
          "estat-navtree-portlet-prod/" + \
          "BulkDownloadListing?file=data%2F/" + \
          dataset + ".tsv.gz"

    with request.urlopen(url) as r:
            with gzip.GzipFile(fileobj=r) as data:
                file_content = data.read() #data read as binary from gzip
    # the approach to make the dataframe from the object with read data:
    # https://stackoverflow.com/questions/39213597/convert-text-data-from-requests-object-to-dataframe-with-pandas
    # the eurostat data contains not strict columns:first line with separator ',',the date columns - separator - '\t'
    df = pd.read_csv(io.StringIO(file_content.decode('utf-8')),sep=",|\t| [^ ]?\t", na_values=":",
                     engine="python",encoding='utf-8')
    #devide one column 'geo\\date'
    df.columns = [x.split('\\')[0].strip(' ') for x in df.columns]
    # use only file_content,not df,it is binary, so use decode()
    #file_content has the first row with the code we need to decode into names using function get_code
    codes=file_content.decode().split('\t')[0].split('\\')[0].split(',')
    for c in codes:
        df[c].replace(get_codes(c),inplace=True)
    return df

In [34]:
def clean_df(dataset_file):
    """ 
    use it if you have downloaded file
    https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?sort=1&dir=dic%2Fen
    """

   
    with gzip.open(dataset_file, 'rb') as f:
        file_content = f.read()
    df = pd.read_csv(io.StringIO(file_content.decode('utf-8')),sep=",|\t| [^ ]?\t", na_values=":",
                     engine="python",encoding='utf-8')
    df.columns = [x.split('\\')[0].strip(' ') for x in df.columns]
    codes=file_content.decode().split('\t')[0].split('\\')[0].split(',')
    for c in codes:
        df[c].replace(get_codes(c),inplace=True)
    return df

In [35]:
get_renew = get_df(dataset='NRG_CB_RW') #Supply, transformation and consumption of renewables and wastes
get_renew

,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,Distribution losses,Primary solid biofuels,Terajoule,Albania,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Distribution losses,Primary solid biofuels,Terajoule,Austria,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Distribution losses,Primary solid biofuels,Terajoule,Bosnia and Herzegovina,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Distribution losses,Primary solid biofuels,Terajoule,Belgium,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Distribution losses,Primary solid biofuels,Terajoule,Bulgaria,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,110.0,40.0,22.0,12.0,13.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34825,Transformation input - patent fuel plants - en...,Non-renewable municipal waste,Terajoule,Slovakia,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34826,Transformation input - patent fuel plants - en...,Non-renewable municipal waste,Terajoule,Türkiye,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34827,Transformation input - patent fuel plants - en...,Non-renewable municipal waste,Terajoule,Ukraine,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34828,Transformation input - patent fuel plants - en...,Non-renewable municipal waste,Terajoule,United Kingdom,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df_get_renew = pd.DataFrame(get_renew)
df_get_renew

,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,Distribution losses,Primary solid biofuels,Terajoule,Albania,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Distribution losses,Primary solid biofuels,Terajoule,Austria,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Distribution losses,Primary solid biofuels,Terajoule,Bosnia and Herzegovina,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Distribution losses,Primary solid biofuels,Terajoule,Belgium,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Distribution losses,Primary solid biofuels,Terajoule,Bulgaria,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,110.0,40.0,22.0,12.0,13.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34825,Transformation input - patent fuel plants - en...,Non-renewable municipal waste,Terajoule,Slovakia,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34826,Transformation input - patent fuel plants - en...,Non-renewable municipal waste,Terajoule,Türkiye,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34827,Transformation input - patent fuel plants - en...,Non-renewable municipal waste,Terajoule,Ukraine,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34828,Transformation input - patent fuel plants - en...,Non-renewable municipal waste,Terajoule,United Kingdom,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
df_get_renew.sort_values(['geo'])

,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,Distribution losses,Primary solid biofuels,Terajoule,Albania,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28380,Transformation input - charcoal production pla...,Pure biogasoline,Thousand tonnes,Albania,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6407,Final consumption - industry sector - non-ferr...,Pure biodiesels,Thousand tonnes,Albania,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28423,Transformation input - charcoal production pla...,Pure biodiesels,Thousand tonnes,Albania,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6364,Final consumption - industry sector - non-ferr...,Pure biogasoline,Thousand tonnes,Albania,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23562,Energy sector - gas works - energy use,Biogases,Terajoule,United Kingdom,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11264,Final consumption - other sectors - commercial...,Renewable municipal waste,Terajoule,United Kingdom,NaN,NaN,263.0,243.0,615.0,283.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1761,Final consumption - energy use,Charcoal,Thousand tonnes,United Kingdom,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23605,Energy sector - gas works - energy use,Geothermal,Terajoule,United Kingdom,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# NaN = 0
# for year in range (2021, 1990):
#     df_get_renew.loc[(df_get_renew.a == 0), '2021':'1990'] = NaN
# df_get_renew
df_get_renew['2020'].where(~(df_get_renew.'2020'=0), other=NaN, inplace=True)


SyntaxError: invalid syntax (1421254821.py, line 5)

In [38]:
df1_get_renew = df_get_renew.dropna()
df1_get_renew

,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
559,Exports,Primary solid biofuels,Terajoule,Albania,241.20,364.300,417.690,721.386,721.386,60.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
560,Exports,Primary solid biofuels,Terajoule,Austria,15705.39,15216.869,14953.044,14351.234,12561.712,11300.87,...,6173.0,5399.0,5307.0,2932.0,2846.0,2300.0,1553.0,2312.0,2212.0,2113.0
562,Exports,Primary solid biofuels,Terajoule,Belgium,0.00,0.000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
563,Exports,Primary solid biofuels,Terajoule,Bulgaria,5366.44,5113.330,5760.679,4950.002,4228.367,4076.00,...,262.0,49.0,430.0,306.0,361.0,248.0,226.0,219.0,0.0,78.0
564,Exports,Primary solid biofuels,Terajoule,Cyprus,0.00,0.000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26566,Change in stock,Non-renewable municipal waste,Terajoule,Serbia,0.00,0.000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26567,Change in stock,Non-renewable municipal waste,Terajoule,Sweden,0.00,0.000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26568,Change in stock,Non-renewable municipal waste,Terajoule,Slovenia,0.00,0.000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26569,Change in stock,Non-renewable municipal waste,Terajoule,Slovakia,0.00,0.000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
df1_get_renew[df_get_renew['siec'].isin(['Inland consumption - calculated'])]

C:\Users\zirav\AppData\Local\Temp\ipykernel_17552\4985869.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1_get_renew[df_get_renew['siec'].isin(['Inland consumption - calculated'])]


,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990


In [40]:
df1_get_renew[df_get_renew['geo'].isin(['Sweden', 'Germany', 'France', 'Denmark', 'Czechia'])]

C:\Users\zirav\AppData\Local\Temp\ipykernel_17552\4050881379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1_get_renew[df_get_renew['geo'].isin(['Sweden', 'Germany', 'France', 'Denmark', 'Czechia'])]


,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
565,Exports,Primary solid biofuels,Terajoule,Czechia,13500.00,12688.939,12539.015,12408.773,11753.894,11397.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
567,Exports,Primary solid biofuels,Terajoule,Denmark,0.00,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
574,Exports,Primary solid biofuels,Terajoule,France,7738.51,7303.379,8520.284,9940.763,9149.431,8814.225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
595,Exports,Primary solid biofuels,Terajoule,Sweden,5390.00,2753.000,2245.000,2951.000,3808.000,4056.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,Exports,Charcoal,Thousand tonnes,Czechia,0.00,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26524,Change in stock,Renewable municipal waste,Terajoule,Sweden,0.00,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26537,Change in stock,Non-renewable municipal waste,Terajoule,Czechia,0.00,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26539,Change in stock,Non-renewable municipal waste,Terajoule,Denmark,0.00,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26546,Change in stock,Non-renewable municipal waste,Terajoule,France,0.00,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# df_get_renew[df_get_renew['geo'].isin(['Greece'])]

In [ ]:
# col = 2021
# while col in range (2021, 1990):
#     df_get_renew['GWh', col] = df_get_renew[col]/3.6
#     col = col - 1
# df_get_renew

In [ ]:
# df_get_renew_pivoted = df_get_renew.pivot(index='siec', columns='geo', values='2021')
# df_get_renew_pivoted

In [41]:
df_get_renew.info

<bound method DataFrame.info of                                                  nrg_bal  \
0                                    Distribution losses   
1                                    Distribution losses   
2                                    Distribution losses   
3                                    Distribution losses   
4                                    Distribution losses   
...                                                  ...   
34825  Transformation input - patent fuel plants - en...   
34826  Transformation input - patent fuel plants - en...   
34827  Transformation input - patent fuel plants - en...   
34828  Transformation input - patent fuel plants - en...   
34829  Transformation input - patent fuel plants - en...   

                                siec       unit  \
0             Primary solid biofuels  Terajoule   
1             Primary solid biofuels  Terajoule   
2             Primary solid biofuels  Terajoule   
3             Primary solid biofuels  Teraj

In [42]:
df_get_renew.columns

Index(['nrg_bal', 'siec', 'unit', 'geo', '2021', '2020', '2019', '2018',
       '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009',
       '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000',
       '1999', '1998', '1997', '1996', '1995', '1994', '1993', '1992', '1991',
       '1990'],
      dtype='object')

In [43]:
get_energy = get_df(dataset='NRG_CB_E')#Supply, transformation and consumption of electricity
get_energy

,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,Available for final consumption,Electricity,Gigawatt-hour,Albania,7030.299,6656.144,6530.243,6480.824,6136.388,5674.000,...,4367.0,2277.0,2148.0,2818.0,2084.0,1875.0,1697.0,1444.0,1341.0,1780.0
1,Available for final consumption,Electricity,Gigawatt-hour,Austria,66084.891,63577.133,66027.931,65855.941,65569.869,64527.283,...,51846.0,50699.0,49884.0,49270.0,47651.0,46538.0,45903.0,45281.0,45767.0,43577.0
2,Available for final consumption,Electricity,Gigawatt-hour,Bosnia and Herzegovina,10843.000,10240.000,11301.000,11820.000,11785.000,11494.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Available for final consumption,Electricity,Gigawatt-hour,Belgium,83660.400,80871.400,83306.400,84263.000,83809.800,83228.100,...,76050.0,75527.0,73321.0,71405.0,69829.0,67822.0,64644.0,63871.0,61783.0,59108.0
4,Available for final consumption,Electricity,Gigawatt-hour,Bulgaria,31684.820,30511.049,31119.715,31389.986,31445.466,30257.000,...,25358.0,27744.0,28524.0,32053.0,30746.0,27875.0,27890.0,27575.0,31230.0,36484.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Slovakia,397.000,387.000,316.000,404.000,418.000,334.000,...,336.0,307.0,295.0,340.0,438.0,348.0,539.0,533.0,540.0,1158.0
2318,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Türkiye,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2319,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Ukraine,0.000,0.000,1834.900,1579.000,1574.200,1633.000,...,246.0,246.0,246.0,281.0,276.0,293.0,277.0,290.0,302.0,293.0
2320,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,United Kingdom,NaN,NaN,2115.259,3390.866,3859.190,4014.340,...,3774.0,2594.0,2477.0,2430.0,2282.0,2051.0,1948.0,2257.0,2109.0,2626.0


In [44]:
df_get_energy = pd.DataFrame(get_energy)
df_get_energy

,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,Available for final consumption,Electricity,Gigawatt-hour,Albania,7030.299,6656.144,6530.243,6480.824,6136.388,5674.000,...,4367.0,2277.0,2148.0,2818.0,2084.0,1875.0,1697.0,1444.0,1341.0,1780.0
1,Available for final consumption,Electricity,Gigawatt-hour,Austria,66084.891,63577.133,66027.931,65855.941,65569.869,64527.283,...,51846.0,50699.0,49884.0,49270.0,47651.0,46538.0,45903.0,45281.0,45767.0,43577.0
2,Available for final consumption,Electricity,Gigawatt-hour,Bosnia and Herzegovina,10843.000,10240.000,11301.000,11820.000,11785.000,11494.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Available for final consumption,Electricity,Gigawatt-hour,Belgium,83660.400,80871.400,83306.400,84263.000,83809.800,83228.100,...,76050.0,75527.0,73321.0,71405.0,69829.0,67822.0,64644.0,63871.0,61783.0,59108.0
4,Available for final consumption,Electricity,Gigawatt-hour,Bulgaria,31684.820,30511.049,31119.715,31389.986,31445.466,30257.000,...,25358.0,27744.0,28524.0,32053.0,30746.0,27875.0,27890.0,27575.0,31230.0,36484.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Slovakia,397.000,387.000,316.000,404.000,418.000,334.000,...,336.0,307.0,295.0,340.0,438.0,348.0,539.0,533.0,540.0,1158.0
2318,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Türkiye,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2319,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Ukraine,0.000,0.000,1834.900,1579.000,1574.200,1633.000,...,246.0,246.0,246.0,281.0,276.0,293.0,277.0,290.0,302.0,293.0
2320,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,United Kingdom,NaN,NaN,2115.259,3390.866,3859.190,4014.340,...,3774.0,2594.0,2477.0,2430.0,2282.0,2051.0,1948.0,2257.0,2109.0,2626.0


In [45]:
df_get_energy.sort_values(['geo'])

,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,Available for final consumption,Electricity,Gigawatt-hour,Albania,7030.299,6656.144,6530.243,6480.824,6136.388,5674.000,...,4367.0,2277.0,2148.0,2818.0,2084.0,1875.0,1697.0,1444.0,1341.0,1780.0
559,Final consumption - industry sector - not else...,Electricity,Gigawatt-hour,Albania,NaN,91.242,90.749,73.707,50.669,40.000,...,43.0,361.0,391.0,741.0,506.0,527.0,473.0,403.0,365.0,482.0
1763,Energy sector - gas-to-liquids plants - energy...,Electricity,Gigawatt-hour,Albania,NaN,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
516,Final consumption - industry sector - non-meta...,Electricity,Gigawatt-hour,Albania,NaN,114.063,113.447,110.000,100.000,99.000,...,196.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1806,Energy sector - gas works - energy use,Electricity,Gigawatt-hour,Albania,NaN,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1632,Energy sector - coke ovens - energy use,Electricity,Gigawatt-hour,United Kingdom,NaN,NaN,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
686,Final consumption - industry sector - transpor...,Electricity,Gigawatt-hour,United Kingdom,NaN,NaN,4568.185,4765.530,4756.283,4847.861,...,5615.0,5586.0,5634.0,6557.0,0.0,0.0,0.0,0.0,0.0,0.0
1675,Energy sector - charcoal production plants - e...,Electricity,Gigawatt-hour,United Kingdom,NaN,NaN,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1718,Energy sector - energy use,Electricity,Gigawatt-hour,United Kingdom,NaN,NaN,6630.311,6815.210,7112.043,7156.219,...,9310.0,9672.0,9870.0,10412.0,9083.0,7518.0,9615.0,9984.0,9794.0,9984.0


In [49]:
df1_get_energy = df_get_energy[df_get_energy['geo'].isin(['Sweden', 'Germany', 'France', 'Denmark', 'Czechia'])]
df1_get_energy

,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
6,Available for final consumption,Electricity,Gigawatt-hour,Czechia,61941.754,59091.845,61188.862,61302.719,60925.147,59762.129,...,51178.000,52196.000,53163.000,54146.000,52155.00,49312.000,47773.000,48148.000,49708.000,53037.000
8,Available for final consumption,Electricity,Gigawatt-hour,Denmark,34436.566,32822.466,32460.086,32119.670,32255.674,32061.184,...,32759.000,32608.000,32487.000,32403.000,31568.00,31728.000,30709.000,30337.000,29829.000,28862.000
15,Available for final consumption,Electricity,Gigawatt-hour,France,442525.466,422058.105,444778.227,448741.212,451970.126,454686.891,...,402878.233,393169.927,381797.334,384223.329,368378.99,362965.677,356206.625,355593.904,346347.728,323068.276
36,Available for final consumption,Electricity,Gigawatt-hour,Sweden,130012.000,125678.000,127372.000,130571.000,130459.000,130614.000,...,128911.000,128818.000,127585.000,128349.000,126736.00,124569.000,123167.000,122000.000,132335.000,130778.000
49,Distribution losses,Electricity,Gigawatt-hour,Czechia,3650.878,4117.058,4299.969,4269.383,4374.745,4080.000,...,4921.000,4953.000,5088.000,5154.000,4768.00,4660.000,4793.000,3860.000,3813.000,3996.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Sweden,58.000,73.000,32.000,58.000,36.000,0.000,...,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000
2285,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Czechia,1573.738,1684.773,1507.800,1361.823,1530.466,1570.000,...,718.000,654.000,517.000,596.000,375.00,436.000,314.000,326.000,326.000,400.000
2287,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Denmark,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000
2294,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,France,1880.923,1975.779,2203.683,2848.863,2962.216,2633.117,...,6298.000,5556.000,5237.000,5836.000,4230.00,3102.000,4187.000,5005.000,5436.000,4942.000


In [50]:
df1_get_energy.sort_values(['geo'])

,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
6,Available for final consumption,Electricity,Gigawatt-hour,Czechia,61941.754,59091.845,61188.862,61302.719,60925.147,59762.129,...,51178.0,52196.0,53163.0,54146.0,52155.0,49312.0,47773.0,48148.0,49708.0,53037.0
1081,Final consumption - transport sector - pipelin...,Electricity,Gigawatt-hour,Czechia,NaN,54.425,48.654,47.341,40.986,31.000,...,33.0,38.0,41.0,52.0,56.0,57.0,54.0,0.0,0.0,0.0
1038,Final consumption - transport sector - not els...,Electricity,Gigawatt-hour,Czechia,NaN,10.696,11.896,11.244,14.237,0.000,...,988.0,1057.0,984.0,1071.0,955.0,1116.0,1161.0,2701.0,2361.0,3167.0
995,Final consumption - transport sector - energy use,Electricity,Gigawatt-hour,Czechia,NaN,1644.099,1767.266,1753.379,1697.558,1636.000,...,2182.0,2311.0,2280.0,2488.0,2383.0,2539.0,2660.0,2701.0,2361.0,3167.0
952,Final consumption - other sectors - not elsewh...,Electricity,Gigawatt-hour,Czechia,NaN,0.000,0.000,0.000,0.000,0.000,...,1575.0,1558.0,1896.0,2119.0,1682.0,1827.0,1893.0,1962.0,1991.0,1914.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,Final consumption - transport sector - not els...,Electricity,Gigawatt-hour,Sweden,NaN,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1025,Final consumption - transport sector - energy use,Electricity,Gigawatt-hour,Sweden,NaN,2757.000,2738.000,2573.000,2476.000,2664.000,...,3016.0,2796.0,2955.0,3067.0,2718.0,2469.0,2340.0,2472.0,2403.0,2474.0
982,Final consumption - other sectors - not elsewh...,Electricity,Gigawatt-hour,Sweden,NaN,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1369,Net electricity production,Electricity,Gigawatt-hour,Sweden,165881.000,160914.000,165645.000,159677.000,160526.000,152531.000,...,150539.0,154385.0,145027.0,136557.0,144226.0,138907.0,142020.0,142533.0,143040.0,142511.0


In [ ]:
df1_get_energy_consumption = df1_get_renew.loc['2020-01-01':'2020-12-02', 'DE_bioenergy_capacity':'DE_wind_onshore_capacity']
df1_get_energy_consumption